# Build Text to SQL with Cortex Analyst

In this notebook we show how to build a text-to-sql app using Cortex Analyst.

## Create the required roles to use Cortex Analyst

Make sure to add your username in Line 10!

In [ ]:
USE WAREHOUSE S;

USE ROLE SECURITYADMIN;

CREATE OR REPLACE ROLE cortex_user_role;
GRANT DATABASE ROLE SNOWFLAKE.CORTEX_USER TO ROLE cortex_user_role;

USE ROLE SECURITYADMIN;

GRANT ROLE cortex_user_role TO USER <user>;

## Create the required databases, schemas and warehouse for this example

In [ ]:
USE ROLE ACCOUNTADMIN;

-- Create demo database
CREATE OR REPLACE DATABASE cortex_analyst_demo;

-- Create schema
CREATE OR REPLACE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Create warehouse
CREATE OR REPLACE WAREHOUSE cortex_analyst_wh
    WAREHOUSE_SIZE = 'large'
    WAREHOUSE_TYPE = 'standard'
    AUTO_SUSPEND = 60
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
COMMENT = 'Warehouse for Cortex Analyst demo';

GRANT USAGE ON WAREHOUSE cortex_analyst_wh TO ROLE cortex_user_role;
GRANT OPERATE ON WAREHOUSE cortex_analyst_wh TO ROLE cortex_user_role;

GRANT OWNERSHIP ON SCHEMA cortex_analyst_demo.revenue_timeseries TO ROLE cortex_user_role;
GRANT OWNERSHIP ON DATABASE cortex_analyst_demo TO ROLE cortex_user_role;

USE ROLE cortex_user_role;

-- Use the created warehouse
USE WAREHOUSE cortex_analyst_wh;

USE DATABASE cortex_analyst_demo;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Create stage for raw data
CREATE OR REPLACE STAGE raw_data DIRECTORY = (ENABLE = TRUE);

/*--
• Fact and Dimension Table Creation
--*/

-- Fact table: daily_revenue
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.daily_revenue (
    date DATE,
    revenue FLOAT,
    cogs FLOAT,
    forecasted_revenue FLOAT,
    product_id INT,
    region_id INT
);

-- Dimension table: product_dim
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.product_dim (
    product_id INT,
    product_line VARCHAR(16777216)
);

-- Dimension table: region_dim
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.region_dim (
    region_id INT,
    sales_region VARCHAR(16777216),
    state VARCHAR(16777216)
);

There are three data files and one YAML file included in the this repository.

daily_revenue.csv
region.csv
product.csv
revenue_timeseries.yaml
You will now upload these files to your Snowflake account and ingest the data files into the tables created in the previous step.

To upload the data files:

1. Set your role to Cortex User Role
2. Navigate to the Data tab in Snowsight, and select Add Data
3. On the Add Data page, select Load files into a stage
4. Select the four files that you want to upload (listed above)
5. Select CORTEX_ANALYST_DEMO as Database, REVENUE_TIMESERIES as Schema, and RAW_DATA as Stage
6. Click Upload

Let's go check that the files were successfully uploaded to the stage. In the Snowsight UI:

7. Select Data » Databases
8. Select the CORTEX_ANALYST_DEMO database and REVENUE_TIMESERIES Schema that contain the stage
9. Select Stages and select the RAW_DATA stage
10. If prompted, select Enable Directory Table and the CORTEX_ANALYST_WH to refresh the directory table

You should see the four files listed in the stage.

## Copy data from stage into tables

In [ ]:
/*--
• looad data into tables
--*/

USE ROLE CORTEX_USER_ROLE;
USE DATABASE CORTEX_ANALYST_DEMO;
USE SCHEMA CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES;
USE WAREHOUSE CORTEX_ANALYST_WH;

COPY INTO CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.DAILY_REVENUE
FROM @raw_data
FILES = ('daily_revenue.csv')
FILE_FORMAT = (
    TYPE=CSV,
    SKIP_HEADER=1,
    FIELD_DELIMITER=',',
    TRIM_SPACE=FALSE,
    FIELD_OPTIONALLY_ENCLOSED_BY=NONE,
    REPLACE_INVALID_CHARACTERS=TRUE,
    DATE_FORMAT=AUTO,
    TIME_FORMAT=AUTO,
    TIMESTAMP_FORMAT=AUTO
    EMPTY_FIELD_AS_NULL = FALSE
    error_on_column_count_mismatch=false
)

ON_ERROR=CONTINUE
FORCE = TRUE ;



COPY INTO CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.PRODUCT_DIM
FROM @raw_data
FILES = ('product.csv')
FILE_FORMAT = (
    TYPE=CSV,
    SKIP_HEADER=1,
    FIELD_DELIMITER=',',
    TRIM_SPACE=FALSE,
    FIELD_OPTIONALLY_ENCLOSED_BY=NONE,
    REPLACE_INVALID_CHARACTERS=TRUE,
    DATE_FORMAT=AUTO,
    TIME_FORMAT=AUTO,
    TIMESTAMP_FORMAT=AUTO
    EMPTY_FIELD_AS_NULL = FALSE
    error_on_column_count_mismatch=false
)

ON_ERROR=CONTINUE
FORCE = TRUE ;



COPY INTO CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.REGION_DIM
FROM @raw_data
FILES = ('region.csv')
FILE_FORMAT = (
    TYPE=CSV,
    SKIP_HEADER=1,
    FIELD_DELIMITER=',',
    TRIM_SPACE=FALSE,
    FIELD_OPTIONALLY_ENCLOSED_BY=NONE,
    REPLACE_INVALID_CHARACTERS=TRUE,
    DATE_FORMAT=AUTO,
    TIME_FORMAT=AUTO,
    TIMESTAMP_FORMAT=AUTO
    EMPTY_FIELD_AS_NULL = FALSE
    error_on_column_count_mismatch=false
)

ON_ERROR=CONTINUE
FORCE = TRUE ;

## Create a Cortex Search service to help Analyst

Now, you will integrate Cortex Search as a way to improve literal string searches to help Cortex Analyst generate more accurate SQL queries. Writing the correct SQL query to answer a question sometimes requires knowing exact literal values to filter on. Since those values can't always be extracted directly from the question, a search of some kind may be needed.

In [ ]:
USE DATABASE cortex_analyst_demo;
USE SCHEMA revenue_timeseries;
use ROLE cortex_user_role;

CREATE OR REPLACE CORTEX SEARCH SERVICE product_line_search_service
  ON product_dimension
  WAREHOUSE = cortex_analyst_wh
  TARGET_LAG = '1 hour'
  AS (
      SELECT DISTINCT product_line AS product_dimension FROM product_dim
  );

## Call Cortex Analyst

First, we'll just show how to make the REST API call and return SQL

In [ ]:
import json
import requests
from typing import List
import _snowflake
from snowflake.snowpark.context import get_active_session

# Change this to match your semantic model stage/path
DEFAULT_SEMANTIC_MODEL = "CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.RAW_DATA/revenue_timeseries.yaml"

API_ENDPOINT = "/api/v2/cortex/analyst/message"
API_TIMEOUT = 50000  # in milliseconds

def get_sql_from_cortex_analyst(query: str) -> List[str]:
    """
    Calls Cortex Analyst with the user's query and returns any generated SQL statements.
    
    Returns:
        A list of SQL statements found in the Analyst's response (there can be more than one).
    """
    # Build the message list (only one user message for simplicity).
    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": query}],
        }
    ]
    
    # Build the request body
    request_body = {
        "messages": messages,
        "semantic_model_file": f"@{DEFAULT_SEMANTIC_MODEL}",
    }
    
    # Make the request to the Analyst API
    resp = _snowflake.send_snow_api_request(
        "POST",  # method
        API_ENDPOINT,  # path
        {},  # headers
        {},  # params
        request_body,  # body
        None,  # request_guid
        API_TIMEOUT,  # timeout in milliseconds
    )
    
    # Use resp.json() to get parsed JSON
    parsed_content = json.loads(resp["content"])

    return parsed_content["message"]["content"][1]["statement"]

user_query = "what is the highest revenue recorded in a single day? which day?"
sql = get_sql_from_cortex_analyst(user_query)

In [ ]:
sql

## Run the SQL

Next, we can run the sql and save it to a variable

In [ ]:
session = get_active_session()

sql_output = session.sql(sql)

sql_output

## Pass SQL output to LLM-readable markdown

Make sure `tabulate` library is installed

In [ ]:
markdown_sql_output = sql_output.to_pandas().to_markdown(index=False)

In [ ]:
markdown_sql_output

## Send the query and SQL output to the LLM

In [ ]:
from snowflake.cortex import Complete
import pandas as pd

messages = [
        {
            'role': 'system',
            'content': 'You are a helpful assistant that sql output to answer natural language questions.'
        },
        {
            'role': 'user',
            'content': f'The user has posed a question, which is captured in: {user_query}. '
                       f'The question has been translated into a SQL statement, executed, and the results are found in:'
                       f'\n{markdown_sql_output}'
                       f'Please answer the user question.'
        }
    ]

options = {
    'guardrails': True,
}

Complete("claude-3-5-sonnet", messages, options = options)

## Put it together

In [ ]:
def answer_question_using_analyst(query: str):
    # use cortex analyst to generate sql for the query
    sql = get_sql_from_cortex_analyst(query)
    # execute sql
    sql_output = session.sql(sql)
    # make the output LLM-readable
    markdown_sql_output = sql_output.to_pandas().to_markdown(index=False)
    # send query and sql results to LLM
    messages = [
        {
            'role': 'system',
            'content': 'You are a helpful assistant that sql output to answer natural language questions.'
        },
        {
            'role': 'user',
            'content': f'The user has posed a question, which is captured in: {user_query}. '
                       f'The question has been translated into a SQL statement, executed, and the results are found in:'
                       f'\n{markdown_sql_output}'
                       f'Please answer the user question.'
        }
    ]
    
    
    options = {
        'guardrails': True,
    }

    response = Complete("claude-3-5-sonnet", messages, options = options)

    return response

In [ ]:
answer_question_using_analyst("what is the highest revenue recorded in a single day? which day?")